# Libraries

In [29]:
# standard
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
from math import sqrt
import time

# reading data
import os
import json
from collections import defaultdict

# machine learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.fft import rfft, irfft, fftn, ifftn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW

# visuals
import matplotlib.pyplot as plt
import seaborn as sns

# measuring ressources
import time
import psutil
import GPUtil
import threading

# eFormer
from eFormer.embeddings import Encoding, ProbEncoding, PositionalEncoding
from eFormer.sparse_attention import ProbSparseAttentionModule, DetSparseAttentionModule
from eFormer.loss_function import CRPS, weighted_CRPS
from eFormer.sparse_decoder import DetSparseDecoder, ProbSparseDecoder
from eFormer.Dataloader import TimeSeriesDataProcessor

# transformer Benchmarks
from Benchmarks.Benchmarks import VanillaTransformer, Informer

%store -r Kelmarsh_df Penmanshiel_df

# Hyperparameters

In [30]:
# set global parameters
hyperparameters = {
    'n_heads': 4,
    'ProbabilisticModel': False,
    # embeddings
    'len_embedding': 64,
    'batch_size': 512,
    # general
    'pred_len': 1,
    'seq_len': 72,
    'patience': 7,
    'dropout': 0.05,
    'learning_rate': 6e-4,
    'WeightDecay': 1e-1,
    'train_epochs': 5,
    'num_workers': 10,
    # benchmarks
    'factor': 1,
    'output_attention': True,
    'd_model': 64,
    'c_out': 6,
    'e_layers': 2,
    'd_layers': 2,
    'activation': 'relu',
    'd_ff': 1,
    }

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

%store hyperparameters

Stored 'hyperparameters' (dict)


# Ressource Management

In [31]:
def check_system_conditions():
    # Get CPU usage for each core
    cpu_percent = round(psutil.cpu_percent(), 4)

    # Get memory information
    memory_info = psutil.virtual_memory()
    memory_used_gb = round(memory_info.used / (1024 ** 3), 4)

    # Get GPU information
    try:
      gpu_info = GPUtil.getGPUs()[0]
      gpu_memory_used_gb = round(gpu_info.memoryUsed / 1024, 4)
    except IndexError:
      # If no GPU is found, set variables to None
      gpu_memory_used_gb = None

    # Collect data in a dictionary
    comp_usage = {
        'CPU Usage': cpu_percent,
        'Memory Usage (GB)': memory_used_gb,
        'GPU Usage (GB)': gpu_memory_used_gb
    }

    return comp_usage

# Load Data

In [32]:
def shifted_data(data, forecast, look_back):
    data = data.set_index('# Date and time')
    data.index.names = [None]
    data = data.drop(['Long Term Wind (m/s)'], axis=1)
    shifts = range(forecast, look_back + forecast)
    variables = data.columns
        
    shifted_columns = []
    for column in variables:
        for i in shifts:
            shifted_df = data[[column]].shift(i)
            shifted_df.rename(columns={column: f"{column} (lag {i})"}, inplace=True)
            shifted_columns.append(shifted_df)
        
    data = data.drop(['Wind speed (m/s)'], axis=1)
    data_shifted = pd.concat([data] + shifted_columns, axis=1)
    data_shifted.dropna(inplace=True)
        
    return data_shifted

In [33]:
data = Kelmarsh_df['1']
data = data.set_index('# Date and time')
data.index.names = [None]
data = data.drop(['Long Term Wind (m/s)'], axis=1)

data.to_csv('../data/Results/test_data.csv')

In [34]:
test_df = shifted_data(Kelmarsh_df['1'], 4, 72)

In [35]:
# Assuming `df` is your initial DataFrame
processor = TimeSeriesDataProcessor(
    dataframe=data,
    forecast=hyperparameters['pred_len'],
    look_back=hyperparameters['seq_len'],
    batch_size=hyperparameters['batch_size'])
    
train_loader, test_loader, eval_loader = processor.create_dataloaders()

In [36]:
for batch in train_loader:
    features, labels = batch
    break

%store features labels

Stored 'features' (Tensor)
Stored 'labels' (Tensor)


In [37]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0
            if self.verbose:
                print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f})')
            self.val_loss_min = val_loss

# Linear Model

In [22]:
# Single Layer Perceptron Model
class LinearModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearModel, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        x = self.linear(x)
        weights = self.linear.weight.data
        
        return x.squeeze(1), weights

# Transformer Model

In [23]:
class eFormer(nn.Module):
    def __init__(self, batch_size, seq_len, in_features, forecast, len_embedding_vector, n_heads, probabilistic_model=False):
        super(eFormer, self).__init__()
        self.probabilistic_model = probabilistic_model
        self.n_heads = n_heads
        self.len_embedding_vector = len_embedding_vector

        # Initialize encoding model
        if probabilistic_model:
            self.encoding_model = ProbEncoding(in_features=in_features, batch_size=batch_size, seq_len=seq_len, len_embedding_vector=len_embedding_vector)
        else:
            self.encoding_model = Encoding(in_features=in_features, batch_size=batch_size, seq_len=seq_len, len_embedding_vector=len_embedding_vector)

        # Initialize attention module
        if probabilistic_model:
            self.attention_module = ProbSparseAttentionModule(d_model=len_embedding_vector, n_heads=n_heads, prob_sparse_factor=5, seq_len=seq_len)
        else:
            self.attention_module = DetSparseAttentionModule(d_model=len_embedding_vector, n_heads=n_heads, prob_sparse_factor=5, seq_len=seq_len)

        # Initialize decoder
        # Assuming the decoder initialization does not actually require the output shape directly but parameters that depend on the model configuration
        if probabilistic_model:
            self.decoder = ProbSparseDecoder(d_model=len_embedding_vector, n_heads=n_heads, batch_size=batch_size, seq_len=seq_len, forecast_horizon=forecast)
        else:
            self.decoder = DetSparseDecoder(d_model=len_embedding_vector, n_heads=n_heads, batch_size=batch_size, seq_len=seq_len, forecast_horizon=forecast)

    def forward(self, features_matrix):
        if torch.isnan(features_matrix).any():
            raise ValueError('NaN values detected in Input')

        embeddings = self.encoding_model(features_matrix)
        if torch.isnan(embeddings).any():
            raise ValueError('NaN values detected in Embeddings')

        encoder_output = self.attention_module(embeddings, embeddings, embeddings)
        if torch.isnan(encoder_output).any():
            raise ValueError('NaN values detected in Sparse Attention Output')

        forecasts, crps_weights = self.decoder(encoder_output)
        return forecasts, crps_weights

In [27]:
# initiate model etc
early_stopping = EarlyStopping(
    patience=hyperparameters['patience'],
    verbose=True
    )
#"""
model = eFormer(
    in_features=(hyperparameters['seq_len'] * 2),
    batch_size=hyperparameters['batch_size'],
    seq_len=hyperparameters['seq_len'],
    forecast=hyperparameters['pred_len'],
    len_embedding_vector=hyperparameters['len_embedding'],
    n_heads=hyperparameters['n_heads'],
    probabilistic_model=hyperparameters['ProbabilisticModel']).to(device)
"""
model = LinearModel(
    input_size=(hyperparameters['seq_len'] * 2),
    output_size=hyperparameters['pred_len'],
).to(device)
"""
optimizer = AdamW(
    params = model.parameters(),
    lr=hyperparameters['learning_rate'],
    weight_decay=hyperparameters['WeightDecay']
    )
loss_fn = nn.MSELoss()
num_epochs = hyperparameters['train_epochs']

# function to run monitoring in a separate thread
def monitor_system_usage(every_n_seconds=10, keep_running=lambda: True, results_list=[]):
    while keep_running():
        comp_usage = check_system_conditions()
        results_list.append(comp_usage)
        time.sleep(every_n_seconds)

# Initialize a list to store the results
system_usage_results = []

# Define a lambda function to control the monitoring loop
keep_monitoring = lambda: keep_monitoring_flag
keep_monitoring_flag = True # Initialize the flag before starting training

# Start the monitoring thread
monitor_thread = threading.Thread(target=monitor_system_usage, args=(5, keep_monitoring, system_usage_results))
monitor_thread.start()

for epoch in range(num_epochs):
    epoch_start_time = time.time()
    model.train()
    train_losses = []
    for features, labels in train_loader:
        features, labels = features.to(device), labels.to(device)
        optimizer.zero_grad()
        predictions, crps_weights = model(features)
        loss = loss_fn(predictions.unsqueeze(1), labels)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

    train_loss_avg = np.mean(train_losses)
    print(f"Epoch {epoch + 1} / {num_epochs} with Loss: {round(train_loss_avg, 6)}")

    # Validation phase
    model.eval()
    validation_losses = []
    predictions_collected = []
    groundtruth_collected = []
    batches_collected = 0  # Counter to keep track of the batches
    with torch.no_grad():
        for features, labels in eval_loader:
            features, labels = features.to(device), labels.to(device)
            predictions, crps_weights = model(features)
            val_loss = loss_fn(predictions, labels)
            validation_losses.append(val_loss.item())

            if batches_collected >= 10:
                pass  # Exit loop after collecting from 10 batches
            else:
                predictions_collected.extend(predictions.tolist())
                groundtruth_collected.extend(labels.tolist())
                batches_collected += 1

    val_loss_avg = np.mean(validation_losses)
    print(f"Validation Loss: {round(val_loss_avg, 6)}")

    epoch_end_time = time.time()
    epoch_duration = epoch_end_time - epoch_start_time
    print(f"Epoch Duration: {round(epoch_duration, 4)}s")

    early_stopping(val_loss_avg)
    if early_stopping.early_stop:
        print("Early stopping")
        break

# After training is done, set the flag to False to stop the monitoring thread
keep_monitoring_flag = False
monitor_thread.join()  # Wait for the monitoring thread to finish

# Convert the results list to a DataFrame
system_usage_df = pd.DataFrame(system_usage_results)

# saving weights

KeyboardInterrupt: 

In [13]:
print(predictions.shape)
print(labels.shape)

torch.Size([256])
torch.Size([256])


# Benchmark Models

In [38]:
class Config:
    def __init__(self, dictionary):
        for key, value in dictionary.items():
            setattr(self, key, value)

## vanilla Transformer

In [39]:
# initiate model etc
early_stopping = EarlyStopping(
    patience=hyperparameters['patience'],
    verbose=True
    )
model = VanillaTransformer(
    configs = Config(hyperparameters)
)
optimizer = AdamW(
    params = model.parameters(),
    lr=hyperparameters['learning_rate'],
    weight_decay=hyperparameters['WeightDecay']
    )
loss_fn = nn.MSELoss()
num_epochs = hyperparameters['train_epochs']

# function to run monitoring in a separate thread
def monitor_system_usage(every_n_seconds=10, keep_running=lambda: True, results_list=[]):
    while keep_running():
        comp_usage = check_system_conditions()
        results_list.append(comp_usage)
        time.sleep(every_n_seconds)

# Initialize a list to store the results
system_usage_results = []

# Define a lambda function to control the monitoring loop
keep_monitoring = lambda: keep_monitoring_flag
keep_monitoring_flag = True # Initialize the flag before starting training

# Start the monitoring thread
monitor_thread = threading.Thread(target=monitor_system_usage, args=(5, keep_monitoring, system_usage_results))
monitor_thread.start()

for epoch in range(num_epochs):
    epoch_start_time = time.time()
    model.train()
    train_losses = []
    for features, labels in train_loader:
        features, labels = features.to(device), labels.to(device)
        optimizer.zero_grad()
        predictions, crps_weights = model(features)
        loss = loss_fn(predictions, labels)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

    train_loss_avg = np.mean(train_losses)
    print(f"Epoch {epoch + 1} / {num_epochs} with Loss: {round(train_loss_avg, 6)}")

    # Validation phase
    model.eval()
    validation_losses = []
    predictions_collected = []
    groundtruth_collected = []
    batches_collected = 0  # Counter to keep track of the batches
    with torch.no_grad():
        for features, labels in eval_loader:
            features, labels = features.to(device), labels.to(device)
            predictions, crps_weights = model(features)
            val_loss = loss_fn(predictions, labels)
            validation_losses.append(val_loss.item())

            if batches_collected >= 10:
                pass  # Exit loop after collecting from 10 batches
            else:
                predictions_collected.extend(predictions.tolist())
                groundtruth_collected.extend(labels.tolist())
                batches_collected += 1

    val_loss_avg = np.mean(validation_losses)
    print(f"Validation Loss: {round(val_loss_avg, 6)}")

    epoch_end_time = time.time()
    epoch_duration = epoch_end_time - epoch_start_time
    print(f"Epoch Duration: {round(epoch_duration, 4)}s")

    early_stopping(val_loss_avg)
    if early_stopping.early_stop:
        print("Early stopping")
        break

# After training is done, set the flag to False to stop the monitoring thread
keep_monitoring_flag = False
monitor_thread.join()  # Wait for the monitoring thread to finish

# Convert the results list to a DataFrame
system_usage_df = pd.DataFrame(system_usage_results)

# saving weights

KeyboardInterrupt: 

# Test Area

In [ ]:
test_tensor = torch

In [ ]:
# Concatenate all DataFrames in the dictionary into a single DataFrame
Kelmarsh_full_df = pd.concat(Kelmarsh_df.values(), ignore_index=True)
Penmanshiel_full_df = pd.concat(Penmanshiel_df.values(), ignore_index=True)

Wind_df = pd.concat([Kelmarsh_full_df, Penmanshiel_full_df], ignore_index=True, sort=False)

Wind_df = Wind_df.drop('date', axis=1)

Wind_df.to_csv('../data/Windturbinen/Wind_df.csv')

# To-Do's

- ground truth and forecasted values in graph as time series
- seperate model for wind and company, one model can't output 2 different results for same equation

- label_length = look_back
- sequence_length -> window size